In [11]:
import logging
from collections import namedtuple
import random
from typing import Callable
from itertools import accumulate, product
from copy import deepcopy
from operator import xor
random.seed(42)
logging.getLogger().setLevel(logging.DEBUG)


In [12]:
Nimply = namedtuple("Nimply", "row, num_objects")

In [13]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k
        self._total_elements = num_rows*num_rows

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    @property
    def k(self) -> int:
        return self._k
    
    @property
    def total_elements(self) -> int:
        return self._total_elements

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

### SUPPORT FUNCTIONS

In [14]:
def nim_sum(state: Nim) -> int:
    *_, result = accumulate(state.rows, xor)
    return result

def cook_status(state: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = [
        (r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1) if state.k is None or o <= state.k
    ]
    cooked["active_rows_number"] = sum(o > 0 for o in state.rows)
    cooked["shortest_row"] = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    cooked["longest_row"] = max((x for x in enumerate(state.rows)), key=lambda y: y[1])[0]
    cooked["random"]=random.choice([r for r, c in enumerate(state.rows) if c > 0])
    cooked["nim_sum"] = nim_sum(state)    
    cooked["completion"] = sum(o for o in state.rows) / state.total_elements
    
    brute_force = list()
    for m in cooked["possible_moves"]:
        tmp = deepcopy(state)
        tmp.nimming(m)
        brute_force.append((m, nim_sum(tmp)))
    cooked["brute_force"] = brute_force
    return cooked

### Benchmarking strategies
Here we use strategy for benchmark more advanced strategies that we designed.

In [15]:
def randomNim() -> Callable:
    def pure_random(state: Nim) -> Nimply:
        """Random row, random number of elements"""
        row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
        num_objects = random.randint(1, state.rows[row])
        return Nimply(row, num_objects)
    return pure_random

def gabrieleNim() -> Callable:
    def gabriele(state: Nim) -> Nimply:
        """Pick always the maximum possible number of the lowest row"""
        possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
        return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))
    return gabriele

def optimalStrategy() -> Callable:
    def optimal_strategy(state: Nim) -> Nimply:
        data = cook_status(state)
        return next((bf for bf in data["brute_force"] if bf[1] == 0), random.choice(data["brute_force"]))[0]
    return optimal_strategy

def randomAllNim() -> Callable:
    def randomAll(state: Nim) -> Nimply:
        row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
        num_objects = random.randint(1, state.rows[row])
        return Nimply(row, num_objects)
    return randomAll

def longestAllNim() -> Callable:
    def longestAll(state: Nim) -> Nimply:
        row =max((x for x in enumerate(state.rows)), key=lambda y: y[1])[0]
        num_objects = state.rows[row]
        return Nimply(row, num_objects)
    return longestAll

#Here the strategy improves on the pure random, where it improves the last move
def randomSmartNim() -> Callable: 
    def randomSmart(state: Nim) -> Nimply:
        data = cook_status(state)
        if data["active_rows_number"]==1:
           return Nimply(data["random"], state.rows[data["random"]])
        else: 
            row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
            num_objects = random.randint(1, state.rows[row])
            return Nimply(row, num_objects)
    return randomSmart

In [16]:
NUM_MATCHES = 1

#This utility helps benchmark strategies
def evaluate(strategy1: Callable, strategy2: Callable, NIM_SIZE: int) -> float:
    opponent = (strategy1, strategy2)
    won = 0
    for m in range(NUM_MATCHES):
        nim = Nim(NIM_SIZE)
        player = 0
        while nim:
            ply = opponent[player](nim)
            logging.debug(f"player: {player} ply: {ply}")
            nim.nimming(ply)
            player = 1 - player
        if player == 1:
            won += 1
    return won / NUM_MATCHES

### 2-params evolutive strategies

In [17]:
def E2longestVSshortest_allVS1smart(genome: dict) -> Callable:
    '''Shortest row vs Longest Row and pick one element vs take the maximum number of elements'''
    def evolvable(state: Nim) -> Nimply:
        data = cook_status(state)
        if data["active_rows_number"]==1:
           return Nimply(data["random"], state.rows[data["random"]])

        if random.random() < genome["p1"]:
            if random.random() < genome["p2"]:
                ply = Nimply(data["longest_row"], state.rows[data["longest_row"]])
            else:
                ply = Nimply(data["longest_row"], 1)
        else:
            if random.random() < genome["p2"]:
                ply = Nimply(data["shortest_row"], state.rows[data["shortest_row"]])
            else:
                ply = Nimply(data["shortest_row"], 1)

        return ply
    return evolvable

def E2randomVSshortest_allVS1smart(genome: dict) -> Callable:
    '''Random row vs Shortest Row and pick one elements vs take the maximum number of elements'''
    def evolvable(state: Nim) -> Nimply:
        data = cook_status(state)
        if data["active_rows_number"]==1:
           return Nimply(data["random"], state.rows[data["random"]])

        if random.random() < genome["p1"]:
            if random.random() < genome["p2"]:
                ply = Nimply(data["random"], state.rows[data["random"]])
            else:
                ply = Nimply(data["random"], 1)
        else:
            if random.random() < genome["p2"]:
                ply = Nimply(data["shortest_row"], state.rows[data["shortest_row"]])
            else:
                ply = Nimply(data["shortest_row"], 1)

        return ply
    return evolvable

def E2longestVSshortest_allVS1allsmart(genome: dict) -> Callable:
    '''Shortest row vs Longest Row and a complex selection of whether to take one element or a subset'''
    def evolvable(state: Nim) -> Nimply:
        data = cook_status(state)
        if data["active_rows_number"]==1:
           return Nimply(data["random"], state.rows[data["random"]])
        if random.random() < genome["p1"]:
            if random.random() < genome["p2"]:
                ply = Nimply(data["longest_row"], state.rows[data["longest_row"]])
            else:
                if(data["completion"]==1):
                    ply= Nimply(data["longest_row"], 1)
                else:
                    ply = Nimply(data["longest_row"], state.rows[data["longest_row"]])
        else:
            if random.random() < genome["p2"]:
                ply = Nimply(data["shortest_row"], state.rows[data["shortest_row"]])
            else:
                if(data["completion"]==1):
                    ply= Nimply(data["shortest_row"], 1)
                else:
                    ply = Nimply(data["shortest_row"], state.rows[data["shortest_row"]])
        return ply
    return evolvable

def E2longestVSrandom_allVS1allsmart(genome: dict) -> Callable:
    '''Random row vs Longest Row and a complex selection of whether to take one element or a subset'''
    def evolvable(state: Nim) -> Nimply:
        data = cook_status(state)
        if data["active_rows_number"]==1:
           return Nimply(data["random"], state.rows[data["random"]])
        if random.random() < genome["p1"]:
            if random.random() < genome["p2"]:
                ply = Nimply(data["longest_row"], state.rows[data["longest_row"]])
            else:
                if(data["completion"]==1):
                    ply= Nimply(data["longest_row"], 1)
                else:
                    ply = Nimply(data["longest_row"], state.rows[data["longest_row"]])
        else:
            if random.random() < genome["p2"]:
                ply = Nimply(data["random"], state.rows[data["random"]])
            else:
                if(data["completion"]==1):
                    ply= Nimply(data["random"], 1)
                else:
                    ply = Nimply(data["random"], state.rows[data["random"]])
        return ply
    return evolvable  

def E2shortestVSrandom_allVS1allsmart(genome: dict) -> Callable:
    '''Random row vs Shortest Row and a complex selection of whether to take one element or a subset'''
    def evolvable(state: Nim) -> Nimply:
        data = cook_status(state)
        if data["active_rows_number"]==1:
           return Nimply(data["random"], state.rows[data["random"]])
        if random.random() < genome["p1"]:
            if random.random() < genome["p2"]:
                ply = Nimply(data["shortest_row"], state.rows[data["shortest_row"]])
            else:
                if(data["completion"]==1):
                    ply= Nimply(data["shortest_row"], 1)
                else:
                    ply = Nimply(data["shortest_row"], state.rows[data["shortest_row"]])
        else:
            if random.random() < genome["p2"]:
                ply = Nimply(data["random"], state.rows[data["random"]])
            else:
                if(data["completion"]==1):
                    ply= Nimply(data["random"], 1)
                else:
                    ply = Nimply(data["random"], state.rows[data["random"]])
        return ply
    return evolvable 

def EsafetySmart(genome: dict) -> Callable:
    '''
        Safety strategy creates based on p1 "safety" nets to fall back on, as the game progresses.
        The idea is that creating rows with 2 elements can help this strategy win against more complex benchmarks as
        it uses the safety nets created at the beginning of the game to outsmart the opponent towards the end of the match 
    '''
    #DICTIONARY OF PARAMETERS: {"p1", "p2"}
    # p1: makes safeties 
    # p2: uses safeties 
    def evolvable(state: Nim) -> Nimply:
        data = cook_status(state)
        
        safety = []
        can_be_safety = []
        row = 0
        for row_val in state.rows:
            if row_val > 2:
                can_be_safety.append(row)
            if row_val < 2 and row_val == 1:
                safety.append(row)
            row += 1
        
        if data["active_rows_number"] == 1:
            # take the whole last row, last move to win
            ply = Nimply(data["longest_row"], state.rows[data["longest_row"]])
        elif len(safety) < genome["p1"] and len(can_be_safety) > 0:
            # need safety, make a safety
            row_choice = random.choice(can_be_safety)
            ply = Nimply(row_choice, state.rows[row_choice] - 2)
        elif data["completion"] < genome["p2"] and len(safety) > 0:
            # use safety
            row_choice = random.choice(safety)
            ply = Nimply(row_choice, 1)
        else:
            # do random move
            ply = Nimply(data["random"], state.rows[data["random"]])
        return ply

    return evolvable 

def E3shortestVSlongest_percentage(genome: dict) -> Callable:
    # DICTIONARY OF PARAMETERS: {"%_taken_longest", "%_taken_shortest", "binary_chance"} 
    # binary chance: chance to taking shortest or longest
    # %_taken_shortest, %_taken_longest : percentage of object to take
    def evolvable(state: Nim) -> Nimply:
        data = cook_status(state)

        if random.random() < genome['binary_chance']:
            x = max(1, int(state.rows[data['shortest_row']]*genome['%_taken_shortest']/100))
            ply = Nimply(data['shortest_row'],random.randint(1, x))    
        else:
            x = max(1, int(state.rows[data['longest_row']]*genome['%_taken_longest']/100))
            ply = Nimply(data['longest_row'], random.randint(1, x)) 
        
        return ply
    return evolvable

### Best parameters and evolution
This is the utility to evaluate and evolve our strategies.

In [18]:
NIM_SIZE = 6

def bestP(strategy: Callable):
    p = [0.05, 0.5]
    increment = [0.05, 0.05]

    bestRes=[0,0]
    
    for i in range (0,100):
        nWin=[]
        p1_pool = [p[0] - increment[0], p[0], p[0] + increment[0]]
        p2_pool = [p[1] - increment[1], p[1], p[1] + increment[1]]
        for p1, p2 in product(p1_pool, p2_pool):
            nWin.append((evaluate(strategy({"p1":p1,"p2":p2}),randomAllNim(),NIM_SIZE), [p1, p2]))

        best = max(nWin, key=lambda k: k[0])
        p = best[1]
        if best[0]>bestRes[0]:
            bestRes=best
        
    logging.debug(f"    nwin= {bestRes[0]}        p={bestRes[1]}")

    return bestRes[1]

In [ ]:
print(bestP(E2longestVSshortest_allVS1smart))

DEBUG:root:    nwin= 0.99        p=[-0.8000000000000002, 1.3000000000000007]


[-0.8000000000000002, 1.3000000000000007]


In [ ]:
evaluate(E2longestVSshortest_allVS1smart({"p1":0,"p2":1}), optimalStrategy(), 5)

0.0

### Tournament

In [ ]:
strategies=[E2longestVSshortest_allVS1smart,E2longestVSshortest_allVS1allsmart,E2randomVSshortest_allVS1smart]
hardcoded=[EsafetySmart]
parameters=[]

for s in strategies:
     parameters.append(bestP(s))

strategies=strategies+hardcoded
parameters.append([1, -0.19999999999999996])

for s in parameters:
     print(s)

In [ ]:
rank=[]
for (s,p) in zip(strategies,parameters):

     nWin=evaluate(s({"p1":p[0],"p2":p[1]}),randomSmartNim(),11)
     rank.append([nWin,s])

nWin=evaluate(E3shortestVSlongest_percentage({'binary_chance': -0.051257059667209226, '%_taken_shortest': 14.22880454715865, '%_taken_longest': 80.97968146702988}),randomSmartNim(),11)
rank.append([nWin,E3shortestVSlongest_percentage])


print(sorted(rank,key=lambda s : -s[0]))


### Genetic evolution

In [ ]:
POPULATION_SIZE = 10
OFFSPRING_SIZE = 5
NUM_ITERATIONS = 1000
def init_population() -> list:
    population = list()
    for _ in range(POPULATION_SIZE):
        param = {'%_taken_longest': random.randint(0, 100), '%_taken_shortest': random.randint(0, 100), 'binary_chance' : random.random()}
        if param not in population:
            population.append((param, evaluate(make_strategy_evol(param),randomSmartNim(),11)))
    return population

def tournament_selection(population) -> dict:
    return population[random.choice(range(len(population)))][0]

def tweak(parameters) -> dict:
    new_param = dict()
    new_param["binary_chance"] = parameters["binary_chance"] + random.gauss(0, 0.1)
    new_param["%_taken_shortest"] = parameters["%_taken_shortest"] + random.gauss(0, 0.1)
    new_param["%_taken_longest"] = parameters["%_taken_longest"] + random.gauss(0, 0.1)
    return new_param

In [ ]:
population = init_population()

for _ in range(1000):
    for __ in range(OFFSPRING_SIZE):
        offspring_pool = list()
        parameters = tournament_selection(population)
        offspring = tweak(parameters)
        o = (offspring, evaluate(E3shortestVSlongest_percentage(offspring), randomSmartNim(), NIM_SIZE))
        if o not in offspring_pool:
            offspring_pool.append(o)
    
    population += offspring_pool
    unique_population = list()
    for p in population:
        if p not in unique_population:
            unique_population.append(p)
    population = unique_population
    population.sort(key=lambda x: x[1], reverse=True)
    population = population[:POPULATION_SIZE]
    logging.debug(f"Iteration {_} : best {population[0][0]} nWin {population[0][1]}")

### 1-param evolution

In [ ]:
def Elongest_allVS1(genome: dict) -> Callable:
    def evolvable(state: Nim) -> Nimply:
        data = cook_status(state)

        if random.random() < genome["p"]:
            ply = Nimply(data["longest_row"], state.rows[data["longest_row"]])           
        else:           
            ply = Nimply(data["longest_row"], 1) 

        return ply
    return evolvable

def Eshortest_allVS1(genome: dict) -> Callable:
    def evolvable(state: Nim) -> Nimply:
        data = cook_status(state)

        if random.random() < genome["p"]:
            ply = Nimply(data["shortest_row"], state.rows[data["shortest_row"]])           
        else:           
            ply = Nimply(data["shortest_row"], 1) 

        return ply
    return evolvable

def Eall_shortestVSlongest(genome: dict) -> Callable:
    def evolvable(state: Nim) -> Nimply:
        data = cook_status(state)

        if random.random() < genome["p"]:
            ply = Nimply(data["shortest_row"], state.rows[data["shortest_row"]])
        else:
            ply = Nimply(data["longest_row"],  state.rows[data["longest_row"]])

        return ply
    return evolvable
   
def Elongest_1allVSALL(genome: dict) -> Callable:
    def evolvable(state: Nim) -> Nimply:
        data = cook_status(state)
        
        if random.random() < genome["p"]:
            if(data["completion"]==1):
                ply= Nimply(data["longest_row"], 1)
            else:
                ply = Nimply(data["longest_row"], state.rows[data["longest_row"]])
        else:
            ply = Nimply(data["longest_row"], state.rows[data["longest_row"]])

        return ply
    return evolvable

def Eshortest_1allVSALL(genome: dict) -> Callable:
    def evolvable(state: Nim) -> Nimply:
        data = cook_status(state)
        
        if random.random() < genome["p"]:
            if(data["completion"]==1):
                ply= Nimply(data["shortest_row"], 1)
            else:
                ply = Nimply(data["shortest_row"], state.rows[data["shortest_row"]])
        else:
            ply = Nimply(data["shortest_row"], state.rows[data["shortest_row"]])

        return ply
    return evolvable

def Erandom_allVS1(genome: dict) -> Callable:
    def evolvable(state: Nim) -> Nimply:
        data = cook_status(state)

        if random.random() < genome["p"]:
            ply = Nimply(data["random"], state.rows[data["random"]])            
        else:
            ply = Nimply(data["random"], 1)

        return ply
    return evolvable

def Erandom_j(genome: dict) -> Callable:
    def evolvable(state: Nim) -> Nimply:
        data = cook_status(state)

        if random.random() < genome["p"]:
            ply = Nimply(data["random"], state.rows[data["random"]])            
        else:
            ply = Nimply(data["random"], 1)

        return ply
    return evolvable

In [ ]:
p = 0.5
nWin = 0
previousNWin = 0
lastAction = 0.1

for i in range (0,100):
    
    nWin=evaluate(Eshortest_allVS1({"p":p}),randomSmartNim(),NIM_SIZE)

    if(nWin>previousNWin):
        p+=lastAction
    elif nWin<previousNWin:
        lastAction=-lastAction
        p+=lastAction

    logging.debug(f"    nwin= {nWin}        p={p}       lastAction={lastAction}")
    previousNWin=nWin
    nWin=0
print(p)

### Tournament

In [ ]:
strategies=[E2longestVSshortest_allVS1smart,E2longestVSshortest_allVS1allsmart,E2randomVSshortest_allVS1smart]
hardcoded=[EsafetySmart]
parameters=[]

for s in strategies:
     parameters.append(bestP(s))

strategies=strategies+hardcoded
parameters.append([2,0.55])

for s in parameters:
     print(s)

In [ ]:
rank=[]
for (s,p) in zip(strategies,parameters):

     nWin=evaluate(s({"p1":p[0],"p2":p[1]}),randomSmartNim(),11)
     rank.append([nWin,s])

nWin=evaluate(E3shortestVSlongest_percentage({'binary_chance': -0.051257059667209226, '%_taken_shortest': 14.22880454715865, '%_taken_longest': 80.97968146702988}),randomSmartNim(),11)
rank.append([nWin,4])


print(sorted(rank,key=lambda s : -s[0]))

#MINMAX

In [26]:
#def won()

#nim.nimming(ply)
def evaluate1match(pl0,pl1,nim,activeRows=0) -> float:
    if nim==0:
        if len(pl0)>len(pl1):
            return 1
        else:
            return -1
    # elif activeRows==1:
    #     if len(pl0)>len(pl1):
    #         return -1
    #     else:
    #         return 1
    else:
        return 0
        

   

In [37]:
def minmax(pl0,pl1,nim,player):
    possible= list()
    if nim:
        data=cook_status(nim)
        possible= data["possible_moves"]
        val=evaluate1match(pl0,pl1,nim,data["active_rows_number"])
        logging.debug(f"player: {player} possible:{possible}")
    
    val=evaluate1match(pl0,pl1,nim)
    if(val!= 0 or not possible):
        logging.debug(f"player: {player} val:{val}")
        return None,val
    evaluations=list()
    for ply in possible:
        nimR=deepcopy(nim)
        nimR.nimming(ply)
        plr0=deepcopy(pl0)
        plr0.append(ply)
        _,val=minmax(pl1,plr0,nimR,1-player)
        evaluations.append((ply,-val))
        if player==0 and val==1:
            break
    return max(evaluations,key=lambda k: k[1])

In [38]:
def minmaxNim() -> Callable:
    def wrapperMinMax(state: Nim) -> Nimply:
        pl0=list()
        pl1=list()
        ply=minmax(pl0, pl1, state,0)[0]
        return (ply[0],ply[1])
    return wrapperMinMax



In [39]:

# best_ply,eval=minmax(pl0,pl1,nim)
# print(best_ply)
# nim.nimming(best_ply)
# nim.nimming(Nimply(random.choice([r for r, c in enumerate(nim.rows) if c > 0]),1))
# best_ply,eval=minmax(pl0,pl1,nim)
# print(best_ply)

evaluate(minmaxNim(),optimalStrategy(),2)

DEBUG:root:player: 0 possible:[(0, 1), (1, 1), (1, 2), (1, 3)]
DEBUG:root:player: 1 possible:[(1, 1), (1, 2), (1, 3)]
DEBUG:root:player: 0 possible:[(1, 1), (1, 2)]
DEBUG:root:player: 1 possible:[(1, 1)]
DEBUG:root:player: 0 val:0
DEBUG:root:player: 1 val:0
DEBUG:root:player: 0 possible:[(1, 1)]
DEBUG:root:player: 1 val:0
DEBUG:root:player: 0 val:0
DEBUG:root:player: 1 possible:[(0, 1), (1, 1), (1, 2)]
DEBUG:root:player: 0 possible:[(1, 1), (1, 2)]
DEBUG:root:player: 1 possible:[(1, 1)]
DEBUG:root:player: 0 val:0
DEBUG:root:player: 1 val:0
DEBUG:root:player: 0 possible:[(0, 1), (1, 1)]
DEBUG:root:player: 1 possible:[(1, 1)]
DEBUG:root:player: 0 val:0
DEBUG:root:player: 1 possible:[(0, 1)]
DEBUG:root:player: 0 val:0
DEBUG:root:player: 0 possible:[(0, 1)]
DEBUG:root:player: 1 val:0
DEBUG:root:player: 1 possible:[(0, 1), (1, 1)]
DEBUG:root:player: 0 possible:[(1, 1)]
DEBUG:root:player: 1 val:0
DEBUG:root:player: 0 possible:[(0, 1)]
DEBUG:root:player: 1 val:0
DEBUG:root:player: 1 possible:

0.0